# MOHID data visualiser

In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
import numba
from numba import cuda
import pandas as pd
import plotly
import plotly.graph_objects as go
import time
from matplotlib.cm import get_cmap
from scipy.spatial.distance import cdist
from WGS import WGS
import os
from datetime import datetime
import concurrent.futures
from shapely.geometry import Polygon, Point
from tqdm import tqdm
from joblib import Parallel, delayed
from matplotlib.gridspec import GridSpec
import pickle
from skgstat import Variogram
from scipy.interpolate import interp2d
import fast_interp
import pathlib
from dateutil import rrule
# from fast_interp import interp2d


def vectorize(v):
    return np.array(v).reshape(-1, 1)

# s1: load operation area
plg_op = pd.read_csv("OP2/OPA_GOOGLE.csv").to_numpy()
plg_op_shapely = Polygon(plg_op)
def is_point_legal(lat, lon):
    point = Point(lat, lon)
    return plg_op_shapely.contains(point)

def checkfolder(folder):
    path = pathlib.Path(folder)
    path.mkdir(parents=True, exist_ok=True)
    # print(folder + "is created")


In [2]:
date_start = "20230303"
date_end = "20230304"
dates = []
for dt in rrule.rrule(rrule.DAILY,dtstart=datetime.strptime(date_start, '%Y%m%d'),
                      until=datetime.strptime(date_end, '%Y%m%d')):
    dates.append(dt.strftime('%Y-%m-%d'))
sd = []
for i in range(len(dates)-1): 
    sd.append(dates[i] + "_" + dates[i+1])

In [3]:
sd

['2023-03-03_2023-03-04']

In [6]:

def plt_mohid(mission_date): 
    mohid_path = "mohid/"
    print(mission_date)
    files = os.listdir(mohid_path)
    files.remove("README.md")
    files.sort()
    ind_date = files.index(mission_date)
    file = files[ind_date] + "/WaterProperties.hdf5" # load latest mohid data
    print("Now it will load the MOHID data...")
    t1 = time.time()
    data = h5py.File(mohid_path + file, 'r')
    grid = data.get('Grid')
    lat = np.array(grid.get("Latitude"))[:-1, :-1].flatten()
    lon = np.array(grid.get("Longitude"))[:-1, :-1].flatten()
    depth = []
    sal_mohid = []
    for i in range(1, 26):
        string_z = "Vertical_{:05d}".format(i)
        string_sal = "salinity_{:05d}".format(i)
        depth.append(np.mean(np.array(grid.get("VerticalZ").get(string_z)), axis = 0))
        sal_mohid.append(np.mean(np.array(data.get("Results").get("salinity").get(string_sal)), axis = 0))
    depth = np.array(depth)
    sal_mohid = np.array(sal_mohid)
    t2 = time.time()
    print("Data is loaded correctly, time consumed: ", t2 - t1)
    res = Parallel(n_jobs=14)(delayed(is_point_legal)(la, lo) for la, lo in zip(lat, lon))
    ind_legal_mohid = np.where(res)[0]
    folderpath = "fig/mohid/" + mission_date + "/"
    checkfolder(folderpath)
    for i in tqdm(range(sal_mohid.shape[0])):
        plt.figure()
        plt.scatter(lon[ind_legal_mohid], lat[ind_legal_mohid], c=sal_mohid[i, :, :].flatten()[ind_legal_mohid], 
                    cmap=get_cmap("BrBG", 10), vmin=20, vmax=33)
        plt.colorbar()
        plt.plot(plg_op[:, 1], plg_op[:, 0], 'k-.')
        plt.title("Salinity field on " + mission_date[:10] + " from MOHID at " + str(i) + " o'clock")
        plt.xlabel("Lon [deg]")
        plt.ylabel("Lat [deg]")
        plt.savefig(folderpath + "P_{:03d}.png".format(i))
#         plt.show()
        plt.close("all")
#         break
# plt.scatter(ym, xm, c=df_mohid[:, 2], cmap=get_cmap("BrBG", 10), vmin=10, vmax=36)
# plt.plot(yplg, xplg, 'r-.')
# plt.axvline(7000)
# plt.axhline(5400)
# plt.colorbar()
# plt.show()


In [7]:
for s in sd:
    plt_mohid(s)

2023-03-03_2023-03-04
Now it will load the MOHID data...
Data is loaded correctly, time consumed:  0.09132075309753418


100%|██████████| 25/25 [00:05<00:00,  4.61it/s]
